Hellan Herrmann Johnson Method
====



Genuine, but not well known method for $4^{th}$ order problems

Hellan 67, Herrmann 67, Johnson 73, Brezzi-Raviart 77, Arnold+Brezzi 85, Comodi 89,
Krendl+Rafetseder+Zulehner 16, Chen+Hu+Huang 16, Braess+Pechstein+JS 20

Is an arbitrary order ($k \geq 0$) mixed discretization method on
possibly curved, mapped triangular elements.

Allows hybridization to obtain a positive definite system matrix,
lowest order case corresponds to the Morley element.

The (newer) TD-NNS method is tightly connected to the (much older) HHJ method.

With D. Braess and A. Pechstein: Prager-Synge type constant-free a posteriori error
estimates using $C^0-DG$ and HHJ


Kirchhoff Plate equation
--

$\DeclareMathOperator{\opdiv}{div}$



Find vertical deflection $w$ such that

$$
\opdiv \opdiv \nabla^2 w = f \qquad \text{ + boundary conditions}
$$

Reduce to second order system with bending moments $\sigma$:

\begin{eqnarray*}
\sigma - \nabla^2 w & = & 0 \\
\opdiv \opdiv \sigma & = & f
\end{eqnarray*}

Variational formulation with $\sigma \in H(\opdiv)^{SYM}$ and $w \in H^1$:

$$
\begin{array}{ccccll}
\int \sigma \tau & + & \int \opdiv \tau \, \nabla w & = & 0 & \forall \, \tau \\[0.5em]
\int \opdiv \sigma \nabla v & & & = & \int f v & \forall \, v
\end{array}
$$

Hellan-Herrmann-Johnson method
---

Find $\sigma_h \in \Sigma_h \subset H(\opdiv \opdiv)$ and $w_h \in W_h \subset H^1$ such that

$$
\begin{array}{ccccll}
\int \sigma_h \tau & + & \sum_T \int_T \opdiv \tau \, \nabla w_h + \int_{\partial T} \tau_{nt} \nabla_t w_h & = & 0 & \forall \, \tau \\[0.5em]
\sum_T \int_T \opdiv \sigma_h \nabla v + \int_{\partial T} \sigma_{nt} \nabla_t v & & & = & \int f v & \forall \, v
\end{array}
$$

This mixed method satisfies the magic discrete kernel inclusion

$$
V_{h,0} \subset V_0
$$

leading to the best-approximation property of the bending moments $\sigma$:

$$
\| \sigma - \sigma_h \|_{L_2} \leq \inf_{\tau_h \in \Sigma_h} \| \sigma - \tau_h \|_{L_2}  + \| f - I_h f \|
$$

$\Sigma_h \subset H(\opdiv \opdiv)$... symmetric matrices, piecewise polynomial, continuous nn-component

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

In [ ]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))

Define product finite element space for $H(\text{div div}) \times H^1$

In [ ]:
order = 2
Sigma = HDivDiv(mesh, order=order)  # dirichlet="left|right|top")
W = H1(mesh, order=order+1, dirichlet=".*")   # "bottom|left|right")
X = Sigma * W

print ("ndof-Sigma:", Sigma.ndof, ", ndof-W:", W.ndof, ", ndof-X:", X.ndof)

$$A(\sigma,u; \tau, v) = a(\sigma,\tau) + b(\sigma,v) + b(\tau, u)$$
with
\begin{eqnarray*}
a(\sigma,\tau) & = & \int_\Omega \sigma \tau \, dx \\
b(\sigma, v) & = & \sum_T \int_T \text{div} \sigma \, \nabla v \, dx - \int_{\partial T} \sigma_n \nabla_t v \, ds
\end{eqnarray*}

In [ ]:
sigma, u = X.TrialFunction()
tau, v = X.TestFunction()

n = specialcf.normal(2)
def tang(u): return u-(u*n)*n

a = BilinearForm(X, symmetric=True)
a += (InnerProduct (sigma, tau) + div(sigma)*grad(v) + div(tau)*grad(u)) * dx
a += (-(sigma*n)*tang(grad(v)) - (tau*n)*tang(grad(u)))*dx(element_boundary=True) 
a.Assemble()

f = LinearForm(X)
f += 500*v*dx
f.Assemble()

gfu = GridFunction(X)
gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec

the vertical deflection:

In [ ]:
Draw (gfu.components[1]);

the bending moments $\sigma_{xx}$:

In [ ]:
Draw (gfu.components[0][0,0], mesh);